# Calligaro Nicolas
## Création des données techniques des bateaux
Readme :\
Ce jupyter se concentre sur récupérer les données technique et produire un DataFrame exploitable.\
Runner l'ensemble du jupyter. vérifier que la dernière cellule et ok\
la dernière cellule est le 'main'

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from sklearn.preprocessing import LabelBinarizer
import pickle
from multiprocessing import Pool
import mptools
html = 'https://www.vendeeglobe.org'
df_clas=pickle.Unpickler(open("df_clas.pkl",'rb')).load()

In [3]:
def get_url_skippers(html):
    """
    Récupère la liste de tous les url des skippers depuis le site de base.
    """
    r = requests.get(f'{html}/fr')
    soup = BeautifulSoup(r.content)
    url = []
    for tag in soup.findAll('a', attrs={'class': 'skippers__slide'}):
        url.append(tag.attrs['href'])
    return url

In [4]:
def get_info_boat (html):
    """
    On scrap la page web et 
    On produit un dictionnaire {détail tech : valeur}
    """
    r = requests.get(html)
    soup = BeautifulSoup(r.content)
    dico={}
    masque = "[0-9]*,[0-9]*"
    masque2 = "[0-9]+"
    #On va produire le champ skipper
    #Il a été vu que le champ skipper n'est pas fiable
    #dico['Skipper']=html.split('/')[-1].replace('-',' ').lower()
    #On créé la cle cle qui contient le nom de famille pour la jointure des 2 DF
    dico['Cle'] = html.split('/')[-1].replace('-',' ').lower().split(' ')[-1]
    for tag in soup.findAll('li', attrs={'class': 'skipper-boat-list-specs-list__item'}) :
        categorie = tag.text.split(':')[0]
        text = tag.text.split(':')[1][1:]
        if categorie == 'Numéro de voile ':
            #dico['N']= (int(re.compile(masque2).search(text).group()))
            #print(dico['N'])
            pass
        elif categorie == 'Architecte ':
            dico['constructeur'] = text
            pass
        elif categorie == 'Chantier ':
            dico['chantier'] = text
            pass
        elif categorie == 'Date de lancement ':
            dico['age'] = 2020-int(text.split(' ')[-1])
            pass
        elif categorie == 'Longueur ':
            dico['longueur'] = float(re.compile(masque).search(text).group().replace(',','.'))
            pass
        elif categorie == 'Largeur ':
            dico['largeur'] = float(re.compile(masque).search(text).group().replace(',','.'))
            pass
        elif categorie == "Tirant d'eau ":
            dico['tirant'] = float(re.compile(masque).search(text).group().replace(',','.'))
            pass
        elif categorie == 'Déplacement (poids) ':
            if text.lower() != 'nc':
                dico['poids'] = float(re.compile(masque2).search(text).group())
            else : 
                #dico['poids'] = 'na'
                pass
        elif categorie == 'Nombre de dérives ':
            if text[:4] == 'foil':
                dico['foils'] = True
                #dico['derive'] = False
            else :
                dico['foils'] = False
                #dico['derive'] = float(re.compile(masque2).search(text).group())
            pass
        elif categorie == 'Hauteur mât ':
            dico['mat'] = int(re.compile(masque2).search(text).group())
            pass
        elif categorie == 'Voile quille ':
            dico['voile_quille'] = text
            pass
        elif categorie == 'Surface de voiles au près ':
            dico['surface'] = int(re.compile(masque2).search(text).group())
            pass
        elif categorie == 'Surface de voiles au portant ':
            dico['surface_portant'] = int(re.compile(masque2).search(text).group())
            pass
    
    return dico

In [5]:
def prod_fiche_tech(html):
    """
    On scrap la page principale pour récupérer l'url de tous les skippers.
    On parcours chaque page du skipper / bateau
    On scrap chaque page du bateau et on concatène le dico associé.
    """
    urls = get_url_skippers(html)
    fiches_tech_boat=[]
    check=len(urls)
    if __name__ ==  '__main__': 
        with Pool() as p :
            fiches_tech_boat = p.map(mptools.Get_Skipper,[f"{html}{url}" for url in urls])        
    print(f"{len(urls)} fiches traitées")
    return fiches_tech_boat

In [6]:
def prod_compl(df):
    """
    On va produire un DF contenant uniquement les colonnes cle/Num_voile/Skipper
    avec des lignes distinct (uniquement 33) en s'appuyant sur le DF du classement
    On fait un join en s'appuyant sur les deux champ clé passé en index.
    on retourne le df_tech compléter du champ Skipper et indexé sur le Num de voile.
    NB : c'est pas génial, mais ça marche ...
    """
    df3 = pd.DataFrame.join(df.set_index('Cle')
                        ,df_clas.drop_duplicates(subset=['Skipper'])[df_clas.columns[[2,3,22]]].set_index('Cle')
                       )
    return df3.set_index(df3.columns[-2]).sort_index()

In [7]:
def get_df_tech(html):
    """
    On produit un premier DF contenant tous les détails technique.
    On corrige quelques NA
    On rajouter les 2 colonnes (Num_Voile et Skipper) manquante
    On utilise LabelBinarizer pour transformer la colonne voile_quille en binaire pour les modèles de prédiction
        Non utilisé mais en prévision
    On rend le dis DF qui sera toujours nommé df_tech dans le reste du tp
    """
    df = pd.DataFrame(prod_fiche_tech(html))
    #On corrige avec la moyenne un champ manquant dans 2 colonnes
    df['poids']=df['poids'].fillna('8')
    df['voile_quille']=df['voile_quille'].fillna('acier forgé')
    #On transforme un champ catégorielle en champ binaire (non utilisé par la suite)
    new_data = LabelBinarizer().fit_transform(df.voile_quille)
    label = sorted(df.voile_quille.unique())
    df[label]=new_data
    #On rajoute 2 colonnes qui corrige des données technique non fiable sur le site.
    df = prod_compl(df)
    #On sauvegarde le DF technique comme objet pour l'utiliser ailleurs
    pickle.Pickler(open("df_tech.pkl",'wb')).dump(df)
    return df

In [8]:
df_tech = get_df_tech(html)
df_tech.shape

33 fiches traitées


(33, 22)